# Running Your PyTorch Projects on RunPod Using a Single Docker Image and GHCR

This guide explains how to prepare your machine-learning projects for RunPod GPU compute using:

* One repository: `PyTorchTutorial`
* Multiple projects stored under:
  `/home/behnam/workspace/PyTorchTutorial/projects/`
* One Dockerfile
* One `requirements.txt`
* GitHub Container Registry (GHCR)
* Automatic dataset download via `kagglehub`

The result is a clean, scalable setup where **one Docker image** can run **any** of your Kaggle and PyTorch projects on RunPod.

---

# 1. Directory structure

The repository is:

```
/home/$USERNAME/workspace/PyTorchTutorial/
```

Inside it, the relevant content for RunPod is only:

```
PyTorchTutorial/
└── projects/
      ├── BrainCancer-MRI/
      ├── brain-mri/
      ├── CIFAR10/
      ├── Lung_Disease_Dataset/
      ├── MNIST/
      ├── segmentation/
      └── visual_odometry/
```

Each project has its own:

* `train.py`
* dataset loaders
* configuration files

RunPod will use only this `/projects` directory.

---

# 2. Create a single `requirements.txt`

This file contains Python dependencies used by **all** projects.

Create:

```
/home/$USERNAME/workspace/PyTorchTutorial/requirements.txt
```

Content:

```
torch
torchvision
torchaudio
timm
opencv-python
numpy
pandas
pyyaml
scikit-learn
albumentations
matplotlib
tqdm
tensorboard
wandb
kagglehub
```

This ensures:

* kagglehub works inside the container
* all ML/vision utilities are available
* all projects can run without installing extra packages

---

# 3. Create a single Dockerfile

Create:

```
/home/$USERNAME/workspace/PyTorchTutorial/Dockerfile
```

Content:

```dockerfile
FROM pytorch/pytorch:2.2.0-cuda12.1-cudnn8-runtime

ENV DEBIAN_FRONTEND=noninteractive

RUN apt-get update && apt-get install -y \
    git wget unzip ffmpeg libgl1 libglib2.0-0 \
    && rm -rf /var/lib/apt/lists/*

WORKDIR /workspace

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Only copy the Kaggle projects folder
COPY projects/ projects/

CMD ["bash"]
```

Notes:

* This image is lightweight.
* Only the `projects/` directory is included in the image.
* Default CMD is `bash`, allowing you to run any project manually.

---

# 4. Build the Docker image locally

Navigate to your repo root:

```bash
cd /home/$USERNAME/workspace/PyTorchTutorial/
```

Build the Docker image:

```bash
docker build -t ghcr.io/behnamasadi/kaggle-projects:latest .
```

You now have a local image ready for upload.

---

# 5. Login to GitHub Container Registry (GHCR)

Generate a GitHub PAT (Personal Access Token) with:

* read:packages
* write:packages

Then login safely:

```bash
echo YOUR_GITHUB_TOKEN | docker login ghcr.io -u behnamasadi --password-stdin
```

This avoids insecure plain-text warnings.

---

# 6. Push the image to GHCR

Execute:

```bash
docker push ghcr.io/behnamasadi/kaggle-projects:latest
```

Your image is now available from anywhere (including RunPod).

---

# 7. RunPod Setup

Go to:

RunPod Dashboard → GPU Cloud → Deploy Pod

Choose:

* Container image:
  `ghcr.io/behnamasadi/kaggle-projects:latest`
* GPU type: RTX 4090 / A6000 / A100 (depending on project)
* Volume: optional (not required because kagglehub downloads automatically)

Launch the pod.

---

# 8. Inside the RunPod terminal

When the pod is running:

Open → Web Terminal

You will see the Docker working directory:

```
/workspace/projects/
```

You can now run any project.

---

# 9. Running individual projects

### Example: Lung Disease Dataset

```bash
cd /workspace/projects/Lung_Disease_Dataset
python train.py
```

### Example: BrainCancer MRI

```bash
cd /workspace/projects/BrainCancer-MRI
python train.py
```

### Example: CIFAR10

```bash
cd /workspace/projects/CIFAR10
python train.py
```

Each project works independently using the same container.

---

# 10. Using kagglehub for automatic dataset download

In each project's `train.py`, you can download datasets like:

```python
import kagglehub
import os

dataset_path = kagglehub.dataset_download("khaleddev/lungs-disease-dataset-broken")
print("Dataset stored at:", dataset_path)
```

This will store data in:

```
/root/.cache/kagglehub/datasets/<dataset-name>/versions/<version>/
```

Notes:

* First run downloads the dataset.
* Subsequent runs reuse cached versions.
* No need to upload datasets to RunPod or Docker.

---

# 11. Optional: project-level dataset symlink

If you want a clean interface inside each project:

```python
os.makedirs("data", exist_ok=True)

local_path = "data/raw"

if not os.path.exists(local_path):
    os.symlink(dataset_path, local_path)
```

Then your dataloader can always use:

```
data/raw/
```

No matter which Kaggle dataset you download.

---

# 12. Logging with TensorBoard or W&B

TensorBoard:

```bash
tensorboard --logdir logs --port 6006
```

Then expose port 6006 in RunPod settings.

Weights & Biases:

* Works automatically if you `wandb login`.

---

# 13. Summary

You now have:

### One repository:

`PyTorchTutorial`

### One Docker image:

```
ghcr.io/behnamasadi/kaggle-projects:latest
```

### One dependency file:

`requirements.txt`

### One folder containing all Kaggle challenges:

`projects/`

### Automatic dataset download with:

`kagglehub.dataset_download()`

### Training on RunPod simply requires:

```
cd /workspace/projects/<project_name>
python train.py
```

This setup is clean, scalable, and GPU-efficient. Perfect for solving many Kaggle challenges from one unified environment.

---


To verify that no training data slipped into `ghcr.io/behnamasadi/kaggle-projects:latest`, you can inspect the image in a few ways:

1. **Check file sizes in the container:**
   ```bash
   docker run --rm -it ghcr.io/behnamasadi/kaggle-projects:latest bash
   ```
   Inside the container:
   ```bash
   du -sh /workspace/projects/* | sort -h
   ```
   Large entries usually mean data/checkpoints were copied.

2. **Find suspicious directories:**
   ```bash
   find /workspace/projects -maxdepth 4 -type d \( -name "data" -o -name "datasets" -o -name "samples" -o -name "checkpoints" \)
   ```
   If these directories exist but should be empty, inspect or remove them.

3. **Confirm `.dockerignore` rules:**
   Ensure paths like `projects/**/data/**`, `projects/**/runs/**`, `projects/**/checkpoints/**`, etc. are ignored. This prevents train/val/test folders, wandb logs, mlruns, and similar artifacts from entering the build context.

4. **Check `docker history`:**
   ```bash
   docker history ghcr.io/behnamasadi/kaggle-projects:latest
   ```
   Look for large layers that might indicate unintended file copies.

If the `.dockerignore` rules are in place and you don’t see unexpectedly large directories inside `/workspace/projects`, you’re safe—only code/configs should be present. Let me know if you want a script to automate the inspection.

Here are the commands to **clean all Docker containers and images** safely and completely.

## 1. Remove **all containers**

First stop all running containers:

```bash
docker stop $(docker ps -aq)
```

Then remove all containers:

```bash
docker rm $(docker ps -aq)
```

## 2. Remove **all images**

```bash
docker rmi $(docker images -aq)
```

## 3. Remove **all volumes**

```bash
docker volume rm $(docker volume ls -q)
```



## 5. Clean everything with a single prune command

This removes **containers, images, networks, build cache** etc.

```bash
docker system prune -a --volumes
```

This is the most complete cleanup.

---

## If you want the shortest version

### Clean EVERYTHING:

```bash
docker system prune -a --volumes
```





### 1. Ensure `.dockerignore` excludes project data
Your repo-level `.dockerignore` already has patterns such as `projects/**/data/**`, `projects/**/checkpoints/**`, etc. After accepting that file, the build context will skip every large folder inside each project (including `segmentation`). Double-check the file contains the block below (order matters so the excludes win):

```
*
!projects/
!projects/**
!requirements.txt
!Dockerfile
!.dockerignore
projects/**/checkpoints/**
projects/**/runs/**
projects/**/wandb/**
projects/**/outputs/**
projects/**/logs/**
projects/**/artifacts/**
projects/**/mlruns/**
projects/**/mlartifacts/**
projects/**/tensorboard/**
projects/**/tb_logs/**
projects/**/data/**
projects/**/datasets/**
projects/**/downloaded_data/**
projects/**/kitti_dataset/**
projects/**/samples/**
```

(Plus the re-inclusion lines for small config files.)

### 2. Rebuild the image with a clean context
Because the previous image was built before these ignore rules existed, you need a fresh build. From `/home/$USER/workspace/PyTorchTutorial`:

```bash
docker build --no-cache -t ghcr.io/behnamasadi/kaggle-projects:latest .
docker push ghcr.io/behnamasadi/kaggle-projects:latest
```

### 3. Verify size in the new container
After rebuilding, run the container and check:

```bash
docker run --rm -it ghcr.io/behnamasadi/kaggle-projects:latest bash
du -sh /workspace/projects/* | sort -h
```

You should now see only code/config footprints—on the order of kilobytes or a few megabytes per project—not hundreds of MB. You can also spot-check:

```bash
find /workspace/projects/segmentation -maxdepth 2 -type d -name 'data' -o -name 'checkpoints' -o -name 'wandb'
```

Those directories should no longer exist inside the image.

If you still notice large artifacts, let me know and we can add more targeted patterns (for example, if there’s a differently named data folder I missed).

You can vet the image right after building it locally, before any push. Handy checks:

- **Build (no push yet):**
  ```
  cd /home/$USER/workspace/PyTorchTutorial
  docker build -t kaggle-projects:test .
  ```

- **Inspect size w/out running:**
  ```
  docker image inspect kaggle-projects:test --format='{{.Size}}'
  docker history --no-trunc kaggle-projects:test
  ```

- **Open a shell inside the image:**
  ```
  docker run --rm -it kaggle-projects:test bash
  ```

- **Check project footprints:**
  ```
  du -sh /workspace/projects/* | sort -h
  ```

- **Look for stray data folders:**
  ```
  find /workspace/projects -maxdepth 4 -type d \
      \( -name data -o -name datasets -o -name checkpoints -o -name runs -o -name wandb \)
  ```

- **See whether `data` dirs are empty:**
  ```
  for d in $(find /workspace/projects -maxdepth 3 -type d -name data); do
      echo "$d"; ls -al "$d";
  done
  ```

If the `du` output stays in KB/MB and the targeted `find` commands return nothing (or only expected lightweight files), you’re safe. When satisfied, tag/push:

```
docker tag kaggle-projects:test ghcr.io/behnamasadi/kaggle-projects:latest
docker push ghcr.io/behnamasadi/kaggle-projects:latest
```

This way you never push a bloated image.